In [31]:
import numpy as np
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
import random
import cv2
import os
import sys
import time

import random
random.seed(4)

from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model

%matplotlib inline

from os import listdir
from xml.etree import ElementTree

ROOT_DIR = os.path.abspath("../") #root directory of the project
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5") #local path to trained weights file
MODEL_DIR = os.path.join(ROOT_DIR, "logs") #directory to save logs and trained model

#import Mask RCNN
sys.path.append(ROOT_DIR)  #to find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN

In [32]:
#path = r'C:\Users\fernando.fincatti\Documents\Insper\measuringMacroNutrients\iteration-10\Mask_RCNN\samples\images_2'
#files = os.listdir(path)
#os.chdir(path)
#i = 261

#for file in files:
#    os.rename(file, str(i)+'.jpg')
#    i = i+1

__Configuration for training on the Kangaroo(Custom)  dataset.
Derives from the base Config class and overrides some values.__

In [33]:

class myMaskRCNNConfig(Config):
    
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config" 

    # set the number of GPUs to use along with the number of images per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes
    # background, hot dog, rice and steak
    NUM_CLASSES = 1+1+1+1
   
    # number of training steps per epoch
    STEPS_PER_EPOCH = 100
    
    # Learning rate
    LEARNING_RATE=0.0007
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

In [34]:
config=myMaskRCNNConfig()

In [46]:
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0007
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [36]:
# initialize the Mask R-CNN model for inference 
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [37]:
model.keras_model.summary()

In [38]:
#n load the weights for COCO
model.load_weights(COCO_MODEL_PATH, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [39]:
amodel.keras_model.summary()

In [40]:
def separate_train_and_val(images_dir, b=0.7):
        images=[]
        for filename in listdir(images_dir):
            image_id = filename[:-4] # extract image id
            images.append(int(image_id)) #add image id
        random.shuffle(images) #suffle images
        test = images[0: int(len(images) * 0.7)]
        val = images[int(len(images) * 0.7): len(images)]
        
        return test, val

In [41]:
test, val = separate_train_and_val(r'C:\Users\fernando.fincatti\Documents\Insper\measuringMacroNutrients\iteration-10\Mask_RCNN\samples\images')

In [42]:
class Food101Dataset(Dataset):
    
    # load the dataset definitions
    def load_dataset(self, dataset_dir, test, val, is_train=True):
        
        #Add classes
        self.add_class("dataset", 1, "hot_dog")
        self.add_class("dataset", 2, "steak")
        self.add_class("dataset", 3, "rice")
        
        #define data locations for images and annotations
        images_dir = dataset_dir + '\\images\\'
        annotations_dir = dataset_dir + '\\annots\\'
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        for filename in listdir(images_dir):
            image_id = filename[:-4] # extract image id
            if is_train and int(image_id) in val: 
                continue
            if not is_train and int(image_id) in test:
                continue
            
            img_path = images_dir + filename # setting image file
            ann_path = annotations_dir + image_id + '.xml' #setting annotations file
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path) #adding images and annotations to dataset
    
    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        tree = ElementTree.parse(filename) #load and parse the file
        root = tree.getroot() #get the root of the document
        boxes = list() #extract each bounding box
        for box in root.findall('.//bndbox'):
            name = box.find('name').text #get class
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
        
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    def load_mask(self, image_id):
        info = self.image_info[image_id] #get details of image
        path = info['annotation'] #define anntation  file location
        boxes, w, h = self.extract_boxes(path) #load XML
       
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            if box[4] == 'hot_dog':
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('hot_dog'))
                
            elif box[4] == 'steak':
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('steak'))
            
            elif box[4] == 'rice':
                masks[row_s:row_e, col_s:col_e, i] = 3
                class_ids.append(self.class_names.index('rice'))
                
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    #Return the path of the image."""
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

In [43]:
# prepare train set
train_set = Food101Dataset()
train_set.load_dataset(r'C:\Users\fernando.fincatti\Documents\Insper\measuringMacroNutrients\iteration-10\Mask_RCNN\samples', test, val, is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# prepare test/val set
test_set = Food101Dataset()
test_set.load_dataset(r'C:\Users\fernando.fincatti\Documents\Insper\measuringMacroNutrients\iteration-10\Mask_RCNN\samples', test, val, is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 249
Test: 107


In [44]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
# train weights (output layers or 'heads')
## train heads with higher lr to speedup the learning
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')
history = model.keras_model.history.history


Starting at epoch 0. LR=0.0007

Checkpoint Path: C:\Users\fernando.fincatti\Documents\Insper\measuringMacroNutrients\iteration-10\Mask_RCNN\logs\maskrcnn_config20211101T1929\mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4 

KeyboardInterrupt: 

In [ ]:
model.get_trainable_layers()

In [ ]:
model_path = 'C:\\Users\\fernando.fincatti\\Documents\\Insper\\measuringMacroNutrients\\iteration-10\\Mask_RCNN\\samples\\mask_rcnn_'  + '.' + str(time.time()) + '.h5'
model.keras_model.save_weights(model_path)

In [ ]:
# class that defines and loads the kangaroo dataset

# prepare config
config = myMaskRCNNConfig()
config.display()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights (mscoco) and exclude the output layers

model.load_weights(model_path, by_name=True)
# train weights (output layers or 'heads')
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

In [ ]:
# load an image
image_id = 2
image = test_set.load_image(image_id)
print(image.shape)
print(test_set.class_ids)
# load image mask
mask, class_ids = test_set.load_mask(image_id)

print(mask.shape)
print(class_ids)
# plot image
pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.4)

pyplot.show()

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = pyplot.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

#Loading the model in the inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')

# loading the trained weights o the custom dataset
model.load_weights(model_path, by_name=True)

In [ ]:
img = load_img("C:\\Users\\fernando.fincatti\\Documents\\Insper\\measuringMacroNutrients\\iteration-10\\Mask_RCNN\\samples\\images\\150.jpg")
img = img_to_array(img)

# detecting objects in the image
result= model.detect([img])
r = result[0]
visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                            test_set.class_names, r['scores'],
                            title="Predictions")